In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
class VariableLengthDataset(Dataset):
    def __init__(self, num_samples=100, max_length=10):
        self.sequences = [torch.tensor(np.random.rand(np.random.randint(1, max_length), 1), dtype=torch.float32) for _ in range(num_samples)]
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, index):
        return self.sequences[index]
    
def collate_fn(batch):
    batch_padded = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True,
    padding_value=0)
    lengths = torch.tensor([len(x) for x in batch], dtype=torch.int64)
    return batch_padded, lengths
    
dataset = VariableLengthDataset(num_samples=200, max_length=20)
data_loader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
# 미니 배치 처리를 위해 앞서 정의한 collate_fn 함수

In [2]:
import torch
import torch.nn as nn
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x, lengths):
        # Pack the sequence for efficient processing
        x_packed = nn.utils.rnn.pack_padded_sequence(x, lengths,
        batch_first=True, enforce_sorted=False)
        _, hidden = self.rnn(x_packed)
        output = self.fc(hidden.squeeze(0))
        return output
# Initialize the model
input_size = 1 # Number of features per time-step in the sequence
hidden_size = 32
output_size = 1
model = SimpleRNN(input_size, hidden_size, output_size)

In [3]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, lengths in data_loader:
        # Assuming the target is the sum of the sequence
        targets = inputs.sum(dim=1)
        # Forward pass
        outputs = model(inputs, lengths)
        loss = criterion(outputs, targets)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 14.8983
Epoch [2/5], Loss: 4.6178
Epoch [3/5], Loss: 3.0110
Epoch [4/5], Loss: 8.2044
Epoch [5/5], Loss: 2.8750


In [5]:
# 연습 2
class VariableLengthNegativeDataset(Dataset):
    def __init__(self, num_samples=100, max_length=10, feature_range=(-1, 1)):
        scale = feature_range[1] - feature_range[0]
        self.sequences = [torch.tensor(scale * (np.random.rand(np.random.randint(1, max_length), 1) - 0.5), dtype=torch.float32) for _ in range(num_samples)]
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, index):
        return self.sequences[index]

In [7]:
targets = inputs.clamp(min=0).sum(dim=1).unsqueeze(1)

In [9]:
# 연습 3
class VariableLengthNegativeDataset(Dataset):
    def __init__(self, num_samples=100, max_length=10, feature_range=(-1, 1)):
        scale = feature_range[1] - feature_range[0]
        self.sequences = [torch.tensor(scale * (np.random.rand(np.random.randint(1, max_length), 1) - 0.5), dtype=torch.float32) for _ in range(num_samples)]
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, index):
        return self.sequences[index]

In [10]:
targets = inputs.clamp(min=0).sum(dim=1).unsqueeze(1)